Nesse momento utilize a base de dados `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv`. Uma breve descrição da base:
- Essa base foi retirada de um serviço de streaming de midia. Os atributos são relativos a perfis de consumo de usuários.
- O objetivo é realizar detecção de *churn* (https://resultadosdigitais.com.br/blog/o-que-e-churn/)
- Possui um registro de 17 semanas de uso e os rótulos estáo na tabela `user-status-after-shrink.csv`, que indicam se ao final do período o usuário cancelou e se manteve assinante.

In [41]:
import pandas as pd
import numpy as np
from os import path

In [42]:
DATAFOLDER = path.join('..','..','data')
DATAFILES = ['weekly-infos-before-shrink.csv', 'user-status-after-shrink.csv']

df = pd.read_csv(path.join(DATAFOLDER, DATAFILES[0]))
rt = pd.read_csv(path.join(DATAFOLDER, DATAFILES[1]))

In [43]:
df.set_index('user', inplace=True)
df.head()

,week,total_sessions,total_mediaids,total_days,total_played,max_played_time,age_without_access,sexo,idade,cidade,...,video_info_time_spent_15_30,video_info_time_spent_30_60,video_info_time_spent_60mais,total_dependents,total_active_dependents,total_played_for_dependents,tipo_de_cobranca,total_cancels,month_subs,assinatura_age
user,,,,,,,,,,,,,,,,,,,,,
6I_eMYSy11HxmmB6ED9J2FxOv78P8dCozF05LloY3pA=,4,16,20,3,371.16,89.61,10,NaN,NaN,NaN,...,37.184983,327.524633,0.627417,0,0,0.0,BOLETO WEB,0,2,45.0
6I_eMYSy11HxmmB6ED9J2FxOv78P8dCozF05LloY3pA=,2,6,10,2,209.94,67.38,10,NaN,NaN,NaN,...,15.445183,139.819783,38.829067,0,0,0.0,BOLETO WEB,0,2,45.0
6I_eMYSy11HxmmB6ED9J2FxOv78P8dCozF05LloY3pA=,5,1,17,1,411.78,411.78,10,NaN,NaN,NaN,...,0.000000,411.649567,0.000000,0,0,0.0,BOLETO WEB,0,2,45.0
6I_eMYSy11HxmmB6ED9J2FxOv78P8dCozF05LloY3pA=,3,11,11,5,337.83,49.30,10,NaN,NaN,NaN,...,166.183917,171.653283,0.000000,0,0,0.0,BOLETO WEB,0,2,45.0
er6jk_5K-IkKH1pH8o8dMjInoLw1rQdRUtdsDRZOVdM=,17,1,1,1,46.07,46.07,-249,M,29.0,Niterói,...,NaN,NaN,NaN,0,0,0.0,DEBITO AUTOMATICO,0,10,179.0


In [39]:
rt.head()

,user,status
0,6I_eMYSy11HxmmB6ED9J2FxOv78P8dCozF05LloY3pA=,assinante
1,er6jk_5K-IkKH1pH8o8dMjInoLw1rQdRUtdsDRZOVdM=,assinante
2,FhptY7RehI3m6MgBH-xprB7jDR5lVTIwXBWeazv1q8Y=,assinante
3,PGr6vROhvnJvsBGcZmWptgIKis1HLPHGaYfMqemd5-s=,assinante
4,glfOIEfq5aaBPdNUOPWX88YWAvvxh3SNKcW0GB8RmoI=,assinante


### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
> Exiba a matriz de correlação!

In [14]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})

ModuleNotFoundError: No module named 'pandas_profiling'

### b) Identicar qual a variáveis parecem influenciar no cancelamento da assinatura.

- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [10]:
# code here